In [82]:
!pip install evaluate

In [83]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [84]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
import numpy as np
import evaluate
import torch

# Dataset uploading

In [85]:
train_set = load_dataset("csv", data_files="drive/MyDrive/Colab Notebooks/train_set.csv")
test_set = load_dataset("csv", data_files="drive/MyDrive/Colab Notebooks/test_set.csv")

train_labels = train_set['train']['complexity']
test_labels = test_set['train']['complexity']

In [95]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
labelEncoder = LabelEncoder()
labelEncoder.fit(train_labels)

def tokenize_data(samples):
    tokenized = tokenizer(samples['code'], truncation=True)
    tokenized['labels'] = labelEncoder.transform(samples['complexity']) if samples['complexity'] else None
    return tokenized

train_set = train_set.map(tokenize_data, batched=True)
eval_set = test_set.map(tokenize_data, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3911 [00:00<?, ? examples/s]

Map:   0%|          | 0/978 [00:00<?, ? examples/s]

# Model

In [87]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=7)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Hyperparameters

In [88]:
training_args = TrainingArguments(output_dir="training_results", eval_strategy="epoch", fp16=True)

# Evaluating

In [89]:
def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Trainer

In [90]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set['train'],
    eval_dataset=eval_set['train'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-90-4f0389b32af6>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [91]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.098559,0.651329
2,1.336500,0.795807,0.736196
3,0.788900,0.744982,0.771984


TrainOutput(global_step=1467, training_loss=0.8928131436359013, metrics={'train_runtime': 401.9071, 'train_samples_per_second': 29.193, 'train_steps_per_second': 3.65, 'total_flos': 2998165844194800.0, 'train_loss': 0.8928131436359013, 'epoch': 3.0})

In [124]:
device = torch.cuda.device("cuda" if torch.cuda.is_available() else "cpu")

In [133]:
test_sample = """class Solution:
    def threeSum(self, nums: List[int]) -> List[List[int]]:
        res = set()
        nums.sort()
        for i in range(len(nums)):
            for j in range(i + 1, len(nums)):
                for k in range(j + 1, len(nums)):
                    if nums[i] + nums[j] + nums[k] == 0:
                        tmp = [nums[i], nums[j], nums[k]]
                        res.add(tuple(tmp))
        return [list(i) for i in res]"""

test_sample = tokenizer(test_sample, return_tensors='pt')
inputs = {key: value.to(model.device) for key, value in test_sample.items()}

In [135]:
trainer.predict(test_dataset=inputs)

KeyError: 0